In [59]:
import jax.numpy as jnp
from jax import jit,grad,hessian
from jax.config import config
config.update("jax_enable_x64", True)

In [ ]:
grad()

In [86]:
def get_gaussianRBF(gamma):
    def f(x,y):
        return jnp.exp(-jnp.sum((x-y)**2)/gamma)
    return f

def laplacian(f,arg_num=0):
    hess=hessian(f,argnums=arg_num)
    def lap(x,y):
        return jnp.trace(hess(x,y))
    return lap



In [87]:
gamma=2
def f(x,y):
    return jnp.exp(-1*jnp.sum((x-y)**2))

hf1=hessian(f,argnums=0)
hf2=hessian(f,argnums=1)

In [88]:
Lf=laplacian(f)

Lf(x,y)

Array(0.00193467, dtype=float64)

In [77]:
x=jnp.array([1.,2.],dtype='float64')
y=jnp.array([0.1,-1],dtype='float64')

def lap1(x,y):
    return jnp.trace(hf1(x,y))

In [89]:
lap1(x,y)

Array(0.00193467, dtype=float64)

In [83]:
jnp.trace(hessian(lap1,argnums=1)(x,y))

Array(0.19083836, dtype=float64)

In [40]:
rbf.apply(x,y)

0.6065306597126334

In [41]:
rbf.apply_lapleft(x,y)

0.0

In [24]:
h=0.1
epsx=np.array([h,0])
epsy=np.array([0,h])

In [25]:
(rbf.apply(x+epsx,y)+rbf.apply(x-epsx,y)-2*rbf.apply(x,y))/(h**2)

-0.4007498106989082

In [26]:
(rbf.apply(x+epsy,y)+rbf.apply(x-epsy,y)-2*rbf.apply(x,y))/(h**2)

-0.0008897269422991625

In [27]:
rbf.apply(x+epsy,y)

0.4819089900902024

In [28]:
rbf.apply(x-epsy,y)

0.5886049696783553